In [1]:
#!pip3 install openpyxl

In [2]:
import pandas as pd
import numpy as np
import warnings
from openpyxl import load_workbook
import re
import folium
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

In [3]:
def get_cell_info(wb, data_sheet, row, col):    
    info = data_sheet.cell(row+1, col+1)
    info_crp = {'bold': 1*(info.font.bold),
                    'italic': 1*(info.font.italic),
                    'locked': info.protection.locked,
                    'indent': info.alignment.indent,
                    'value': info.value
                   }
        
    return info_crp

In [4]:
def get_data(wb, data_sheet, freezed_row, label_col, data_cols, columns_):
    
    data_clean = []
    indent = 0
    r = ''

    end_row = freezed_row  
    
    while True:
        try:
            row_ = ['']*(3 + len(data_cols))
            cell_reg_name = get_cell_info(wb, data_sheet,
                          end_row, label_col)
            cell_population = [get_cell_info(wb, data_sheet, 
                          end_row, i) for i in data_cols]
            
            indent = cell_reg_name['indent']
            #Получаем название региона 
            if indent == 0:
                if (cell_reg_name['bold']==1):
                    r = cell_reg_name['value']
                    row_[0] = 0
                    row_[1] = r
                    row_[2] = r
                    for n,i in enumerate(cell_population):
                        row_[n+3] = i['value']
                else:
                    m = cell_reg_name['value']
                    row_[0] = 1
                    row_[1] = r
                    row_[2] = m
                    for n,i in enumerate(cell_population):
                        row_[n+3] = i['value']
                if (row_[-1] != '')&(row_[-1] != np.nan)&(row_[-1] is not None):
                    data_clean.append(row_) 
                else:
                    break
            end_row+=1
        except Exception as e:
            print(e)
            break
    result = pd.DataFrame(data_clean)
    columns = ['level', 'group', 'region']
    columns.extend(columns_)
    result.columns = columns        
    return result

# Данные по населению

In [5]:
years = [str(i) for i in range(2000, 2021)] #доступны данные за этот период
label_col_idx = 0 #номер столбца с названием региона
#data_col_idx = [years.index(str(2020)) + 1] #данные за определенный год
data_col_idxs = [i+1 for i in range(len(years))] #данные за весь период

In [6]:
wb = load_workbook(filename = './data/2021/Раздел 1 - Население.xlsx')
info = pd.read_excel('./data/2021/Раздел 1 - Население.xlsx')
info.head(3)

,Unnamed: 0,Unnamed: 1
0,Социально-экономические показатели по субъекта...,NaN
1,1. НАСЕЛЕНИЕ,NaN
2,1.1.,Численность населения


In [7]:
data_sheet = wb.get_sheet_by_name('1.1.')
freezed_row = int(data_sheet.freeze_panes[1])
df_population = get_data(wb, data_sheet, freezed_row-1, label_col_idx, data_col_idxs, years)
print(df_population.shape[0])
df_population.head()

90


,level,group,region,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Центральный федеральный округ,Центральный федеральный округ,38175.1,38068.5,37946.8,37965.1,38044,38109,38183.4,...,38537.6,38678.9,38819.9,38951.5,39104.3,39209.6,39311.4,39378.1,39433.6,39251.0
1,1,Центральный федеральный округ,Белгородская область,1507.0,1508.1,1511.9,1513.9,1511.7,1511.7,1514.2,...,1536.1,1541,1544.1,1547.9,1550.1,1552.9,1549.9,1547.4,1549.2,1541.3
2,1,Центральный федеральный округ,Брянская область,1407.9,1391.4,1375.0,1360.2,1344.1,1327.7,1312.7,...,1264.4,1253.6,1242.6,1233,1225.8,1220.5,1211,1200.2,1192.5,1182.7
3,1,Центральный федеральный округ,Владимирская область,1558.0,1539.2,1520.1,1509.6,1497.6,1486.5,1475.9,...,1431.9,1421.7,1413.3,1405.6,1397.2,1389.6,1378.3,1365.8,1358.4,1342.1
4,1,Центральный федеральный округ,Воронежская область,2422.4,2397.1,2374.4,2367.4,2364.9,2360.9,2353.8,...,2331.5,2330.4,2328.9,2331.1,2333.5,2335.4,2333.8,2327.8,2324.2,2305.6


In [8]:
df_population['group'] = df_population['group'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))
df_population['region'] = df_population['region'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))
df_population[years] = \
df_population[years].apply(lambda x: x.fillna(0)\
       .map(lambda y: float(y) if (re.fullmatch('([0-9]*)|([0-9]*\.[0-9]*)', str(y)) is not None) else 0))

In [9]:
df_population.dtypes.values

array([dtype('int64'), dtype('O'), dtype('O'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64')], dtype=object)

# Данные о животных

In [10]:
wb = load_workbook(filename = './data/2021/Раздел 13 -Сельское хозяйство.xlsx')
info = pd.read_excel('./data/2021/Раздел 13 -Сельское хозяйство.xlsx')
info[info['Unnamed: 1'].isin(['Поголовье крупного рогатого скота', 'Поголовье свиней',
       'Поголовье овец и коз'])]['Unnamed: 0'].unique()

array(['13.25.', '13.26.', '13.27.'], dtype=object)

In [11]:
years = [str(i) for i in range(2000, 2021)] #доступны данные за этот период
label_col_idx = 0 #номер столбца с названием региона
#data_col_idx = [years.index(str(2020)) + 1] #данные за определенный год
data_col_idxs = [i+1 for i in range(len(years))] #данные за весь период

## Крупный рогатый скот

In [12]:
data_sheet = wb.get_sheet_by_name('13.25.')
freezed_row = int(data_sheet.freeze_panes[1])
df_krs = get_data(wb, data_sheet, freezed_row-1, label_col_idx, data_col_idxs, years)
print(df_krs.shape[0])

91


In [13]:
df_krs.head()

,level,group,region,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Российская Федерация,Российская Федерация,27519.8,27390.2,26846.1,25091.1,23153.8,21625,21561.6,...,19900.8,19679.8,19272.6,18919.9,18620.9,18346.1,18294.2,18151.394,18126.003,18027.2
1,0,Центральный федеральный округ,Центральный федеральный округ,5527.4,5419.3,5215,4619.8,4118.3,3749.3,3549.9,...,2836.4,2843.4,2823.2,2820.7,2860.4,2877.3,2908.4,2977.756,3037.31,3123.5
2,1,Центральный федеральный округ,Белгородская область,459.2,470.6,471.1,410.6,357.3,325.2,317.3,...,235.1,232.7,226.7,221,223,225.4,223.9,231.662,236.132,234
3,1,Центральный федеральный округ,Брянская область,329.3,325.3,318,281.1,249.5,236.3,226.9,...,213.3,250.1,332.5,405.6,424.1,450.3,462.6,491.203,482.535,505
4,1,Центральный федеральный округ,Владимирская область,214.5,206.2,198.5,174.2,159.1,152.4,148.8,...,145.1,141.7,141.6,134.7,135,133.4,133.8,135.983,135.233,133.3


In [14]:
df_krs[years] = \
    df_krs[years].apply(lambda x: x.fillna(0)\
       .map(lambda y: float(y) if (re.fullmatch('([0-9]*)|([0-9]*\.[0-9]*)', str(y)) is not None) else 0))
df_krs['group'] = df_krs['group'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))
df_krs['region'] = df_krs['region'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))

In [15]:
df_krs.dtypes.values

array([dtype('int64'), dtype('O'), dtype('O'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64')], dtype=object)

In [16]:
df_krs.isna().sum().max()

0

## Свиньи

In [17]:
data_sheet = wb.get_sheet_by_name('13.26.')
freezed_row = int(data_sheet.freeze_panes[1])
df_pigs = get_data(wb, data_sheet, freezed_row-1, label_col_idx, data_col_idxs, years)
print(df_pigs.shape[0])

91


In [18]:
df_pigs.head()

,level,group,region,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Российская Федерация,Российская Федерация,15824.4,16227,17600.6,16278.2,13717.2,13811.7,16184.9,...,17262.9,18785.4,19010.3,19451.6,21405.5,21924.6,23075.5,23726.56,25163.165,25850.1
1,0,Центральный федеральный округ,Центральный федеральный округ,3171.9,3225.4,3466.1,3004,2568.6,2661.1,3288.5,...,6032.1,7673.7,8266.8,8704.1,9647.8,10294.2,11336.8,11834.729,12918.29,13417.5
2,1,Центральный федеральный округ,Белгородская область,469.1,490.3,529.8,504.4,470.6,534.6,838.6,...,2700,3304.4,3481.5,3678.2,3954.4,4137.4,4362.8,4526.801,4542.381,4553.6
3,1,Центральный федеральный округ,Брянская область,172.4,172.1,173.4,150.1,126.9,120,123,...,187.5,261,304.7,288.5,282.2,272.5,295.5,306.113,313.844,487
4,1,Центральный федеральный округ,Владимирская область,113.4,117,116.2,106.3,107.3,103.3,112.4,...,141.2,159.7,155.1,145.1,157.4,83.8,43.6,21.787,2.356,3


In [19]:
df_pigs[years] = \
    df_pigs[years].apply(lambda x: x.fillna(0)\
       .map(lambda y: float(y) if (re.fullmatch('([0-9]*)|([0-9]*\.[0-9]*)', str(y)) is not None) else 0))
df_pigs['group'] = df_pigs['group'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))
df_pigs['region'] = df_pigs['region'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))

In [20]:
df_pigs.dtypes.values

array([dtype('int64'), dtype('O'), dtype('O'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64')], dtype=object)

In [21]:
df_pigs.isna().sum().max()

0

## Овцы и козы

In [22]:
data_sheet = wb.get_sheet_by_name('13.27.')
freezed_row = int(data_sheet.freeze_panes[1])
df_sheeps = get_data(wb, data_sheet, freezed_row-1, label_col_idx, data_col_idxs, years)
print(df_sheeps.shape[0])

91


In [23]:
df_sheeps.head()

,level,group,region,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Российская Федерация,Российская Федерация,14961.9,15572.9,16370.3,17261.3,18077.7,18581.4,20194.5,...,22726.9,23998.9,24131.4,24445.4,24606.5,24716.9,24389.1,23129.296,22617.586,21659.9
1,0,Центральный федеральный округ,Центральный федеральный округ,1064.8,1032.8,992.2,931.4,872.8,780.3,814.9,...,990.9,1053.5,1072.2,1110.6,1166.3,1172.4,1128.2,1050.489,1011.034,1034.1
2,1,Центральный федеральный округ,Белгородская область,63.9,60.4,58.5,55.1,52.4,51.9,52.9,...,91.1,104,108,99.8,98.7,101,98.4,80.628,75.114,70
3,1,Центральный федеральный округ,Брянская область,32.7,34.9,35.3,33.1,31.6,30,36.9,...,33.3,31.7,29,29.4,31.1,32.1,29.8,25.857,24.717,25.7
4,1,Центральный федеральный округ,Владимирская область,48.5,45.8,44.3,37.3,34.2,30.2,26.2,...,21.9,21,22.7,21.9,24.8,25.8,28.4,26.511,24.577,20.9


In [24]:
df_sheeps[years] = \
    df_sheeps[years].apply(lambda x: x.fillna(0)\
       .map(lambda y: float(y) if (re.fullmatch('([0-9]*)|([0-9]*\.[0-9]*)', str(y)) is not None) else 0))
df_sheeps['group'] = df_sheeps['group'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))
df_sheeps['region'] = df_sheeps['region'].map(lambda x: re.sub(' +', ' ', x).strip()).map(lambda x: re.sub(r'[^\w\s]','',x))

In [25]:
df_sheeps.dtypes.values

array([dtype('int64'), dtype('O'), dtype('O'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64')], dtype=object)

In [26]:
df_sheeps.isna().sum().max()

0

# На душу населения

Людей и животных Росстат считает примерно похожим образом: численность населения измеряется в тысячах человек; поголовье КРС, свиней и овец/коз - в тысячах голов.

## Расчёт

In [27]:
df_final = df_population\
    .merge(df_krs, on = ['level', 'group', 'region'], suffixes = ['', '_krs'], how = 'left')\
    .merge(df_pigs, on = ['level', 'group', 'region'], suffixes = ['', '_pigs'], how = 'left')\
    .merge(df_sheeps, on = ['level', 'group', 'region'], suffixes = ['', '_sheeps'], how = 'left')

In [28]:
df_final = df_final[df_final['level']==1]

In [29]:
for y in years:
    n = y + '_animals'
    df_final[n] = df_final[y+'_krs'] + df_final[y+'_pigs'] + df_final[y+'_sheeps']

In [30]:
krs_cols = [y + '_krs' for y in years]
pigs_cols = [y + '_pigs' for y in years]
sheeps_cols = [y + '_sheeps' for y in years]
animal_cols = [y + '_animals' for y in years]

In [31]:
df_final.set_index(['level', 'group', 'region'], inplace = True)

In [32]:
krs_shares = np.divide(*df_final[krs_cols].align(df_final[years], axis=0))
krs_shares.columns = [i + '_pc' for i in krs_cols]

pigs_shares = np.divide(*df_final[pigs_cols].align(df_final[years], axis=0))
pigs_shares.columns = [i + '_pc' for i in pigs_cols]

sheeps_shares = np.divide(*df_final[sheeps_cols].align(df_final[years], axis=0))
sheeps_shares.columns = [i + '_pc' for i in sheeps_cols]

animals_shares = np.divide(*df_final[animal_cols].align(df_final[years], axis=0))
animals_shares.columns = [i + '_pc' for i in animal_cols]

## Результат

### На 2020 год

In [33]:
krs_shares[krs_shares['2020_krs_pc']>=1][['2020_krs_pc']].sort_values('2020_krs_pc', ascending = False)

,,,2020_krs_pc
level,group,region,
1,Южный федеральный округ,Республика Калмыкия,1.332222


In [34]:
pigs_shares[pigs_shares['2020_pigs_pc']>=1][['2020_pigs_pc']].sort_values('2020_pigs_pc', ascending = False)

2020_pigs_pc
level group                            region                            
1     Центральный федеральный округ    Белгородская область      2.954389
      СевероЗападный федеральный округ Псковская область         2.328120
      Центральный федеральный округ    Курская область           2.060374
                                       Тамбовская область        1.174678

In [35]:
sheeps_shares[sheeps_shares['2020_sheeps_pc']>=1][['2020_sheeps_pc']].sort_values('2020_sheeps_pc', ascending = False)

2020_sheeps_pc
level group                              region                                        
1     Южный федеральный округ            Республика Калмыкия                   7.160370
      Сибирский федеральный округ        Республика Тыва                       3.698245
      СевероКавказский федеральный округ КарачаевоЧеркесская Республика        2.384401
      Сибирский федеральный округ        Республика Алтай                      1.715385
      СевероКавказский федеральный округ Республика Дагестан                   1.446973
      Южный федеральный округ            Астраханская область                  1.281820

In [36]:
animals_shares[animals_shares['2020_animals_pc']>=1][['2020_animals_pc']].sort_values('2020_animals_pc', ascending = False)

2020_animals_pc
level group                              region                                         
1     Южный федеральный округ            Республика Калмыкия                    8.529630
      Сибирский федеральный округ        Республика Тыва                        4.299031
      Центральный федеральный округ      Белгородская область                   3.151625
      СевероКавказский федеральный округ КарачаевоЧеркесская Республика         2.718522
      Сибирский федеральный округ        Республика Алтай                       2.664253
      СевероЗападный федеральный округ   Псковская область                      2.486295
      Центральный федеральный округ      Курская область                        2.338532
      СевероКавказский федеральный округ Республика Дагестан                    1.745316
      Южный федеральный округ            Астраханская область                   1.578874
      Центральный федеральный округ      Тамбовская область                     1.332663
                                         Орловская область                      1.161446
      Приволжский федеральный округ      Республика Мордовия                    1.092683
      Сибирский федеральный округ        Республика Хакасия                     1.003759

# Карта

In [39]:
geolocator = Nominatim(user_agent="tolko sprosit")

In [40]:
list_of_regions = [i[2] for (i, n) in pd.DataFrame([animals_shares[i]>=1 for i in animals_shares.columns])\
                        .sum(axis = 0)\
                      .items() if n>0]

In [41]:
coordinates = {}
for r in tqdm(list_of_regions):
    #a bit of hardcode
    if r=='КарачаевоЧеркесская Республика':
        r_fixed = 'Карачаево - Черкесская Республика'
        coordinates.update({r: [geolocator.geocode(r_fixed).latitude, \
                                       geolocator.geocode(r_fixed).longitude]})
    else:
        coordinates.update({r: [geolocator.geocode(r).latitude, \
                                       geolocator.geocode(r).longitude]})

In [42]:
coordinates

{'Белгородская область': [50.7080119, 37.5837615],
 'Курская область': [51.6568453, 36.4852695],
 'Орловская область': [52.8392765, 36.4251709],
 'Тамбовская область': [52.9019574, 41.3578918],
 'Псковская область': [57.5358729, 28.8586826],
 'Республика Калмыкия': [46.9017116, 45.249161],
 'Астраханская область': [47.1878186, 47.608851],
 'Республика Дагестан': [43.0883099, 47.14988017944228],
 'КарачаевоЧеркесская Республика': [43.7368326, 41.7267991],
 'Ставропольский край': [44.8632577, 43.4406913],
 'Республика Мордовия': [54.4419829, 44.4661144],
 'Республика Алтай': [50.7114101, 86.8572186],
 'Республика Тыва': [51.4017149, 93.8582593],
 'Республика Хакасия': [53.4399379, 90.0664303],
 'Забайкальский край': [52.248521, 115.956325]}

In [43]:
year = 2020

In [44]:
russia_map = folium.Map(
    location = [geolocator.geocode('Тюмень').latitude, geolocator.geocode('Тюмень').longitude], 
    tiles='cartodbpositron',
    zoom_start = 3.3)

for r in coordinates.keys():
    cow_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f42e.png',
                                          icon_size=(40, 40))
    pig_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f437.png',
                                          icon_size=(40, 40))
    sheep_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f40f.png',
                                          icon_size=(40, 40))
    animals_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f984.png',
                                          icon_size=(40, 40))
    if r in krs_shares[krs_shares[str(year) + '_krs_pc']>=1].reset_index()['region'].values:
        icon_ = cow_icon
    elif r in pigs_shares[pigs_shares[str(year) + '_pigs_pc']>=1].reset_index()['region'].values:
        icon_ = pig_icon
    elif r in sheeps_shares[sheeps_shares[str(year) + '_sheeps_pc']>=1].reset_index()['region'].values:
        icon_ = sheep_icon
    elif r in animals_shares[animals_shares[str(year) + '_animals_pc']>=1].reset_index()['region'].values:
        icon_ = animals_icon
    else:
        icon_ = ''
    
    if icon_ != '':
        folium.Marker(coordinates[r],
                  icon=icon_
                 ).add_to(russia_map)
    
russia_map